In [38]:
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
from generate_robot_instruction import encode_task_generation_prompt, generate_task_data, encode_instruct_prompt, generate_instruction_following_chat_data, post_process_chat_response

In [5]:
message = encode_task_generation_prompt()
message

[{'role': 'user',
  'content': "You are serving as a task-generation helper for a given robot environment.\xa0\n\nHere is the information about the environment. The environment is called {Two Block World}\xa0\nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are two blocks, {block A} and {block B}, with randomized sizes and density in the environment.\nThe blocks are initialized at a random position on a table.\nContents in the {} are the names of the objects in the environment.\n\nCome up with 10 different tasks for the robot to perform.\n\nGenerate the response following the template below:\n### Task {i}: {task description}\nwhere {i} is the task number and {task description} is the task description.\n\n\nAn example task description is: {pick up the heavier block and place it on top of the lighter block}\n\nThe rules for task description:\n1. Only include the objects in the environment in the task description.\n2. The task description doesn't need to include all the o

In [6]:
output = openai.ChatCompletion.create(
    model="gpt-4",
    messages=message,
    temperature=1.0,
    top_p=1,
    max_tokens=512,)
response = output['choices'][0]['message']['content']

In [19]:
output.choices

[<OpenAIObject at 0x7f8dd9558ef0> JSON: {
   "index": 0,
   "message": {
     "role": "assistant",
     "content": "### Task 1: Pick up block B and place it directly next to block A without altering the initial orientation of block A*.\n### Task 2: Manipulate block A and block B such that they are on the opposite sides of the table*.\n### Task 3: Maximize the height by stacking the two blocks on top of each other*.\n### Task 4: Pick up the lighter block and place it on the ground beneath the table without touching the heavier block.\n### Task 5: Rotate block A 90 degrees without moving it from its initial position*.\n### Task 6: Identify which block is denser, then lift that block and place it on top of the other block*.\n### Task 7: Reorient blocks A and B such that they are at 180 degrees from each other with the heaviest block on the table and the lighter block on top of it*.\n### Task 8: Move both block A and block B to the corners of the table.\n### Task 9: Arrange blocks A and B 

In [21]:
import re
import string

In [31]:
def post_process_task_response(response):
    if response is None:
        return []
    ### Parse the response into list of tasks ###
    raw_tasks = re.split("###", response)
    print(raw_tasks)
    tasks = []
    for idx, task in enumerate(raw_tasks):
        # # if the decoding stops due to length, the last example is likely truncated so we discard it
        # if idx == len(raw_tasks) - 1 and response["finish_reason"] == "length":
        #     continue
        ##### Parse the response into task #####
        splitted_data = re.split(f"{idx}:\s+", task)
        if len(splitted_data) != 2:
            continue
        else:
            task = splitted_data[1].strip()

        ##### FILTER OUT Negative Examples #####
        # filter out too short or too long tasks
        if len(task.split()) <= 3 or len(task.split()) > 150:
            continue
        # filter based on keywords that are not suitable for language models.
        # filter those starting with punctuation
        if task[0] in string.punctuation:
            continue
        # filter those starting with non-english character
        if not task[0].isascii():
            continue
        tasks.append(task)
    return tasks

In [ ]:
generate_task_data(num_tasks_to_generate=20)

In [114]:
rst = generate_instruction_following_chat_data(num_instructions_to_generate=1)

### [[{'role': 'user', 'content': 'You are asked to come up with a set of 10 task instructions and corresponding responses. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.\n\nThe instruction and response pairs are happening between the robot and a chatbot guider in a robotic environment.\n\nHere is the information about the {environment}. \n\nThe environment is called "Two Block World" \nThere is a 7DOF Franka robot with a parallel gripper. \nThere are 2 blocks, {block A} and {block B} with randomized sizes and density in the environment.\nThe blocks are initialized at a random position on a table.\nContents in the {} are the names of the objects in the environment.\n\nThe robot is given a long horizon task: {TASK}. \nThe instructions and responses happen when the robot is trying to solve this specific {TASK}. \n\nEach instruction data pair consists of three parts: {instruction}, {input}, {output}\nThe {instruct





prompts: 100%|██████████| 1/1 [01:24<00:00, 84.76s/it]

9it [01:24,  9.42s/it]                       

Request 0 took 84.76s
Generated 9 instructions, kept 9 instructions


In [56]:
seed_tasks_path="./prompts/seeded_tasks.jsonl"
seed_example_path="./prompts/seeded_example.jsonl"
function_file_path="./prompts/skill_functions.jsonl"

In [57]:
import json
seed_tasks = [json.loads(l) for l in open(seed_tasks_path, "r")]
seed_instructions = [json.loads(l) for l in open(seed_example_path, "r")]
functions = [json.loads(l) for l in open(function_file_path, "r")]

In [58]:
batch_input = []
for _ in range(1):
    # only sampling from the seed tasks
    # prompt_instructions = random.sample(seed_instruction_data, num_prompt_instructions)
    prompt = encode_instruct_prompt(
        tasks=seed_tasks,
        functions=functions,
        examples=seed_instructions,
    )
    batch_input.append(prompt)

In [59]:
import utils
decoding_args = utils.OpenAIChatDecodingArguments(
    temperature=1.0,
    top_p=1.0,
    max_tokens=2048,
    stop=["\n21", "21."],
)
chatcompletions = utils.openai_chatcompletion(
    prompts=batch_input,
    model_name="gpt-4",
    decoding_args=decoding_args,
    )

### [[{'role': 'user', 'content': 'You are asked to come up with a set of 20 task instructions and corresponding responses. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.\n\nThe instruction and response pairs are happening between the robot and a chatbot guider in a robotic environment.\n\nHere is the information about the {environment}. \n\nThe environment is called "Two Block World" \nThere is a 7DOF Franka robot with a parallel gripper. \nThere are 2 blocks, {block A} and {block B} with randomized sizes and density in the environment.\nThe blocks are initialized at a random position on a table.\nContents in the {} are the names of the objects in the environment.\n\nThe robot is given a long horizon task: {TASK}. \nThe instructions and responses happen when the robot is trying to solve this specific {TASK}. \n\nEach instruction data pair consists of three parts: {instruction}, {input}, {output}\nThe {instruct

prompts: 100%|██████████| 1/1 [02:14<00:00, 134.13s/it]


In [111]:
instructions = []
for completion in chatcompletions:
    new_instructions = post_process_chat_response(completion)
    instructions += new_instructions

In [113]:
len(instructions)

14

In [104]:
import re
import string
response = completion
raw_instructions = response["message"]["content"]
raw_instructions = re.split("###", raw_instructions)

for idx, inst in enumerate(raw_instructions):
    # if the decoding stops due to length, the last example is likely truncated so we discard it
    if idx == len(raw_instructions) - 1 and response["finish_reason"] == "length":
        continue
    ##### Parse the response into instruction, input, output #####
    idx += 1
    splitted_data = re.split(f"(<Task>|<Instruction>|<Input>|<Output>)", inst)
    print(len(splitted_data))
    if len(splitted_data) != 9:
        continue
    else:
        task = splitted_data[2].strip()
        inst = splitted_data[4].strip()
        input = splitted_data[6].strip()
        input = "" if input.lower() == "<noinput>" else input
        output = splitted_data[8].strip()
        # parse output into <verbal output> and <action output>
        output_splitted_data = re.split(
            f"(\[verbal\]|\[action\])", output
        )
        print(output_splitted_data)
        verbal_output = output_splitted_data[2].strip()
        action_output = output_splitted_data[4].strip()
        ##### FILTER OUT Negative Examples #####
        # filter out too short or too long instructions
        if len(inst.split()) <= 3 or len(inst.split()) > 150:
            continue
        # filter based on keywords that are not suitable for language models.
        # filter those starting with punctuation
        if inst[0] in string.punctuation:
            continue
        # filter those starting with non-english character
        if not inst[0].isascii():
            continue
        instructions.append(
            {
                "task": task,
                "instruction": inst,
                "input": input,
                "verbal_output": verbal_output,
                "action_output": action_output,
            }
        )

1
9
['', '[verbal]', ' Based on the density, block A is lighter. The strategy will be to pick up block A and then place it at the position [1.75, 2.75, 0.75].\n', '[action]', ' reach([1.12, 1.45, 0.75]), grasp(blockA), place(blockA, [1.75, 2.75, 0.75])']
9
['', '[verbal]', ' Yes, the robot should be able to reach block A.\n', '[action]', ' reach([1.12, 1.45, 0.75])']
9
['', '[verbal]', ' The robot should reach towards block A, grasp it, bring it over block B, then place it on top.\n', '[action]', ' reach([2, 4, 0.75]), grasp(blockA), place(blockA, [2.5, 3.75, 1.2])']
9
['', '[verbal]', ' The robot should reach towards block A, grasp it and then shake it. Shaking will be simulated by moving the grasped block back and forth.\n', '[action]', ' reach([1.75, 2.5, 0.45]), grasp(blockA), move_to([1.75, 2.5, 0.65]), move_to([1.75, 2.5, 0.45])']
9
['', '[verbal]', ' The lighter block, which is Block A, should be placed at the position [1.75, 2.75, 0.75]. \n', '[action]', ' place(blockA, [1.75, 

In [105]:
instructions[0]

{'instruction': 'Based on the initial observation, which is the lighter block and what is the strategy to move it to the opposite end of the table?',
 'input': '[[1.12, 1.45, 0.75], [1.75, 2.75, 0.45], [0.0, 0.0, 3]]',
 'verbal_output': 'Based on the density, block A is lighter. The strategy will be to pick up block A and then place it at the position [1.75, 2.75, 0.75].',
 'action_output': 'reach([1.12, 1.45, 0.75]), grasp(blockA), place(blockA, [1.75, 2.75, 0.75])'}